In [12]:
import requests
import datetime
import json
import telegram

#공공 오픈API를 이용하여 외환 정보 받아오기...
def api_set(day):
    date=day.strftime('%Y-%m-%d')
    api_key='kPcffQwkKLScfAOUl8LYW2Y4sG2IVsDi'
    api='https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?authkey={key}&searchdate={date}&data={data}'.format(key=api_key, date=date, data='AP01')
    # data AP01: 환율, AP02: 대출금리, AP03: 국제금리
    req=requests.get(api)
    get_data=req.text
    currency_json=json.loads(get_data)
    
    return currency_json

#코드가 처음부터 시작되는 영역
def main():
    today=datetime.date.today() #오늘 날짜를 datetime형으로 가져오기
    yesterday=today-datetime.timedelta(1) #오늘 정보가 없을 경우 어제 정보를 가져오기 위해 어제 날짜를 datetime형으로 가져오기
    
    result=['*** 오늘 {day}의 환율입니다. ***\n주말 혹은 당일 정보 업데이트가 되지 않으면 전일(주말의 경우 금요일)의 정보가 표시 됩니다.\n'.format(day=today)] #최종 표시될 메시지
    cur_list={'AED':0,'AUD':1,'BHD':2,'BND':3,'CAD':4,'CHF':5,'CNH':6,'DKK':7,'EUR':8,'GBP':9,'HKD':10,'IDR':11,'JPY':12,'KRW':13,'KWD':14,'MYR':15,'NOK':16,'NZD':17,'SAR':18,'SEK':19,'SGD':20,'THB':21,'USD':22}
    #API에서 제공하는 외환 리스트
    
    currency=('USD', 'CNH', 'JPY', 'EUR') #읽어드릴 외환 리스트
    
    if not api_set(today): #오늘 날짜를 입력했을때 API가 빈 정보를 반환할 경우 어제 날짜를 입력하여 정보 가져옴
        raw=api_set(yesterday)
    else: #API 반환 정보가 비어있지 않을 경우 당일 날짜를 입력하여 정보 가져옴
        raw=api_set(today)
        
    for curr in currency: #currency 변수로 설정해둔 외환 리스트의 정보를 가져오고 가공
        num = cur_list[curr]
        data=(raw[num]['cur_nm'], raw[num]['cur_unit'], raw[num]['deal_bas_r'])
        draft='현재 {cur_name}({cur_unit}의 기준 환율은 {currency}원입니다.'.format(cur_name=data[0], cur_unit=data[1], currency=data[2])
        result.append(draft)
        
    text='\n'.join(result)  #챗봇으로 보내기 좋게 메시지 가공
    print(text)
    
    #my_token='telegram'  #botfather로 받은 토큰 입력
    #chatbot=telegram.Bot(token=my_token)
    #chat_id='chatid or channel address' #채널의 경우 @채널주소
    #chatbot.sendMessage(chat_id=chat_id, text=text) #최종 결과물의 text 변수를 챗봇으로 발송
    
if __name__ == '__main__':
    main()
        

*** 오늘 2020-01-16의 환율입니다. ***
주말 혹은 당일 정보 업데이트가 되지 않으면 전일(주말의 경우 금요일)의 정보가 표시 됩니다.

현재 미국 달러(USD의 기준 환율은 1,158.8원입니다.
현재 위안화(CNH의 기준 환율은 168원입니다.
현재 일본 옌(JPY(100)의 기준 환율은 1,054.37원입니다.
현재 유로(EUR의 기준 환율은 1,292.24원입니다.
